# Volt Analytics

## Import Dependencies

In [1]:
import os
import datetime as dt
import numpy as np
import pandas as pd
import pygsheets

#authorization
gc = pygsheets.authorize(service_file='volt-metrics-creds.json')

pd.set_option('display.max_rows', 500)

In [2]:
currentDate =  dt.datetime.today().date()
currentDate

datetime.date(2018, 10, 22)

## Read in CSV files

In [3]:
#Volt Daily Data
voltDailyData = pd.read_csv('data/voltdailydata_' + currentDate.strftime('%Y%m%d') + '.csv')
voltDailyData['Date'] = pd.to_datetime(voltDailyData['Date'])
voltDailyData['Date'] = voltDailyData['Date'].dt.date

In [4]:
voltDailyData.head()

,Date,MilesDriven,EvMilesDriven,GallonsBurned
0,2017-06-25,163.7,52.6,3.03005
1,2017-06-26,202.4,42.5,4.15543
2,2017-06-27,202.3,53.8,3.97314
3,2017-06-28,197.7,52.4,3.93617
4,2017-06-29,192.6,45.2,3.83577


In [5]:
# Calculate total ICE (Internal Combustion Engine) miles driven
voltDailyData = voltDailyData.rename(columns = {'MilesDriven':'TotalMilesDriven'})
voltDailyData['TotalICEMiles'] = voltDailyData['TotalMilesDriven'] - voltDailyData['EvMilesDriven']
voltDailyData.head()

,Date,TotalMilesDriven,EvMilesDriven,GallonsBurned,TotalICEMiles
0,2017-06-25,163.7,52.6,3.03005,111.1
1,2017-06-26,202.4,42.5,4.15543,159.9
2,2017-06-27,202.3,53.8,3.97314,148.5
3,2017-06-28,197.7,52.4,3.93617,145.3
4,2017-06-29,192.6,45.2,3.83577,147.4


In [6]:
# Create new DailyMPG Column, round values after adding new column
voltDailyData['DailyMPG'] = voltDailyData['TotalMilesDriven'] / voltDailyData['GallonsBurned']
voltDailyData['DailyMPG'] = pd.to_numeric(voltDailyData['DailyMPG'], errors='coerse')
voltDailyData = voltDailyData.replace(np.nan, 0)
voltDailyData = voltDailyData.round(2)
voltDailyData.head()

,Date,TotalMilesDriven,EvMilesDriven,GallonsBurned,TotalICEMiles,DailyMPG
0,2017-06-25,163.7,52.6,3.03,111.1,54.03
1,2017-06-26,202.4,42.5,4.16,159.9,48.71
2,2017-06-27,202.3,53.8,3.97,148.5,50.92
3,2017-06-28,197.7,52.4,3.94,145.3,50.23
4,2017-06-29,192.6,45.2,3.84,147.4,50.21


In [7]:
# Add '250+' miles to any DailyMPG value that has a value of infinity
# Why 250+? That's the "infinity mileage" that the volt displays in car
voltDailyData = voltDailyData.replace(np.inf, 250)
voltDailyData.tail()

,Date,TotalMilesDriven,EvMilesDriven,GallonsBurned,TotalICEMiles,DailyMPG
328,2018-10-17,132.7,38.8,2.86,93.9,46.34
329,2018-10-18,124.4,0.0,3.40,124.4,36.56
330,2018-10-19,0.0,0.0,0.00,0.0,0.00
331,2018-10-20,0.0,0.0,0.00,0.0,0.00
332,2018-10-21,0.0,0.0,0.00,0.0,0.00


In [8]:
voltDailyData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 6 columns):
Date                333 non-null object
TotalMilesDriven    333 non-null float64
EvMilesDriven       333 non-null float64
GallonsBurned       333 non-null float64
TotalICEMiles       333 non-null float64
DailyMPG            333 non-null float64
dtypes: float64(5), object(1)
memory usage: 15.7+ KB


## Volt Readings

In [9]:
#Volt Readings throughout the day (usually two), convert Timestamp to datetime object
voltReadings = pd.read_csv('data/voltdata_' + currentDate.strftime('%Y%m%d') + '.csv')
voltReadings.rename(columns = {'LocalTimestamp':'Date', 'GallonsBurned': 'LifeTimeGalBurned'}, inplace=True)
voltReadings['Date'] = pd.to_datetime(voltReadings['Date'])
voltReadings.head()

,Date,BatteryLevel,ChargeState,OilLife,TirePressure_RF,TirePressure_RR,TirePressure_LF,TirePressure_LR,LifeTimeGalBurned,LifetimeFuelEcon,LifetimeEvMiles,LifetimeMiles,LastTripFuelEcon,LastTripEvMiles,LastTripMiles,GallonsOfGas
0,2018-10-22 01:45:01,0.0,NaN,0.0,0.0,0.0,0.0,0.0,1043.88094,58.98207,23002.55251,61570.2607,0.0,0.0,0.0,2.36434
1,2018-10-21 13:52:11,0.0,NaN,0.0,0.0,0.0,0.0,0.0,1043.88094,58.98207,23002.55251,61570.2607,0.0,0.0,0.0,2.40132
2,2018-10-21 01:44:06,0.0,NaN,0.0,0.0,0.0,0.0,0.0,1043.88094,58.98207,23002.55251,61570.2607,0.0,0.0,0.0,2.40132
3,2018-10-20 13:48:50,0.0,NaN,0.0,0.0,0.0,0.0,0.0,1043.88094,58.98207,23002.55251,61570.2607,0.0,0.0,0.0,2.40132
4,2018-10-20 01:48:40,0.0,NaN,0.0,0.0,0.0,0.0,0.0,1043.88094,58.98207,23002.55251,61570.2607,0.0,0.0,0.0,2.40132


In [10]:
# Select only columns desired
voltReadings = voltReadings[['Date', 'LifeTimeGalBurned', 'LifetimeFuelEcon', 'LifetimeEvMiles','LifetimeMiles']]

In [11]:
# Calculate LifetimeICEMiles
voltReadings['LifetimeICEMiles'] = voltReadings['LifetimeMiles'] - voltReadings['LifetimeEvMiles']

In [12]:
# Sort based on timestamp, reorder columns, round values
voltReadings.sort_values(by='Date', inplace=True)
voltReadings = voltReadings[['Date', 'LifeTimeGalBurned','LifetimeFuelEcon','LifetimeEvMiles','LifetimeICEMiles','LifetimeMiles']]
voltReadings = voltReadings.round(2)
voltReadings['Date'] = voltReadings.Date.dt.normalize()
voltReadings.head()

,Date,LifeTimeGalBurned,LifetimeFuelEcon,LifetimeEvMiles,LifetimeICEMiles,LifetimeMiles
739,2017-06-24,593.83,68.35,17941.91,22643.54,40585.45
738,2017-06-25,593.83,68.35,17941.91,22643.54,40585.45
737,2017-06-25,596.65,68.27,17988.50,22746.30,40734.80
736,2017-06-26,596.86,68.27,17994.46,22754.70,40749.16
735,2017-06-26,601.02,68.14,18036.93,22914.62,40951.55


In [13]:
# Drop dupes based on date column
voltReadings.drop_duplicates(subset='Date', keep='last', inplace=True)
voltReadings.head()

,Date,LifeTimeGalBurned,LifetimeFuelEcon,LifetimeEvMiles,LifetimeICEMiles,LifetimeMiles
739,2017-06-24,593.83,68.35,17941.91,22643.54,40585.45
737,2017-06-25,596.65,68.27,17988.50,22746.30,40734.80
735,2017-06-26,601.02,68.14,18036.93,22914.62,40951.55
733,2017-06-27,604.19,68.05,18084.05,23032.74,41116.79
731,2017-06-28,608.17,67.93,18130.94,23180.56,41311.50


In [14]:
# Combine dataframes based on date indexes
voltStats = pd.concat([voltReadings.set_index('Date'),voltDailyData.set_index('Date')], join='inner', axis=1)
voltStats.tail()

,LifeTimeGalBurned,LifetimeFuelEcon,LifetimeEvMiles,LifetimeICEMiles,LifetimeMiles,TotalMilesDriven,EvMilesDriven,GallonsBurned,TotalICEMiles,DailyMPG
Date,,,,,,,,,,
2018-10-17,1039.47,59.08,23002.55,38409.07,61411.62,132.7,38.8,2.86,93.9,46.34
2018-10-18,1041.14,59.04,23002.55,38468.26,61470.82,124.4,0.0,3.40,124.4,36.56
2018-10-19,1043.88,58.98,23002.55,38567.71,61570.26,0.0,0.0,0.00,0.0,0.00
2018-10-20,1043.88,58.98,23002.55,38567.71,61570.26,0.0,0.0,0.00,0.0,0.00
2018-10-21,1043.88,58.98,23002.55,38567.71,61570.26,0.0,0.0,0.00,0.0,0.00


In [15]:
#Remove rows that show 0 for TotalMilesDriven, EvMilesDriven, GallonsBurned, TotalICEMiles, DailyMPG (Days not driven)
voltStats = voltStats[voltStats.DailyMPG != 0]
voltStats.tail()

,LifeTimeGalBurned,LifetimeFuelEcon,LifetimeEvMiles,LifetimeICEMiles,LifetimeMiles,TotalMilesDriven,EvMilesDriven,GallonsBurned,TotalICEMiles,DailyMPG
Date,,,,,,,,,,
2018-10-12,1028.07,59.30,22963.72,37996.41,60960.14,199.9,0.0,5.07,199.9,39.39
2018-10-15,1033.45,59.18,22963.72,38196.40,61160.12,128.2,0.0,3.84,128.2,33.40
2018-10-16,1035.13,59.14,22963.72,38251.18,61214.90,123.3,0.0,3.23,123.3,38.19
2018-10-17,1039.47,59.08,23002.55,38409.07,61411.62,132.7,38.8,2.86,93.9,46.34
2018-10-18,1041.14,59.04,23002.55,38468.26,61470.82,124.4,0.0,3.40,124.4,36.56


In [16]:
# Check info on columns
voltStats['Date'] = voltStats.index
voltStats.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 141 entries, 2017-06-25 to 2018-10-18
Data columns (total 11 columns):
LifeTimeGalBurned    141 non-null float64
LifetimeFuelEcon     141 non-null float64
LifetimeEvMiles      141 non-null float64
LifetimeICEMiles     141 non-null float64
LifetimeMiles        141 non-null float64
TotalMilesDriven     141 non-null float64
EvMilesDriven        141 non-null float64
GallonsBurned        141 non-null float64
TotalICEMiles        141 non-null float64
DailyMPG             141 non-null float64
Date                 141 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(10)
memory usage: 13.2 KB


In [17]:
# Write voltStats DF to CSV
voltStats.to_csv('output/voltstats.csv', encoding='utf-8')

In [18]:
#open the google spreadsheet (where 'VoltStats' is the name of my sheet)
sh = gc.open('VoltStats')

#select the first sheet 
wks = sh[0]
voltStats['Date'] = voltStats['Date'].dt.strftime('%Y-%m-%d')

#update the first sheet with df, starting at cell B2. 
wks.set_dataframe(voltStats,(1,1))